In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random
from IPython.display import display
import re

# 1) Create a Session with a random or fixed User-Agent
s = requests.Session()
s.headers.update({
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/109.0.0.0 Safari/537.36"
    )
})

def get_soup(session, url):
    """Fetch a URL, return BeautifulSoup or None on failure."""
    try:
        resp = session.get(url, timeout=10)
        # Check status code
        if resp.status_code == 200:
            return BeautifulSoup(resp.text, 'html.parser')
        elif resp.status_code == 429:
            print("Received 429. Too many requests. Backing off.")
            # Wait longer or do an exponential backoff
            time.sleep(60)
            return None
        else:
            print(f"Error: got status {resp.status_code} for {url}")
            return None
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

def get_team_links(soup):
    """Extract team links from the league table soup."""
    tables = soup.select('table.stats_table')
    if not tables:
        print("No stats_table found!")
        return []
    league_table = tables[0]
    links = league_table.find_all('a')
    hrefs = [l.get('href') for l in links if l.get('href')]
    hrefs = [h for h in hrefs if 'squads' in h]
    team_urls = [f"https://fbref.com{h}" for h in hrefs]
    return team_urls

def get_player_links(session, team_url):
    soup = get_soup(session, team_url)
    if not soup:
        return []
    tables = soup.select('table')
    if not tables:
        return []
    player_table = tables[0]
    links = player_table.find_all('a')
    hrefs = [l.get('href') for l in links if l.get('href')]
    hrefs = [h for h in hrefs if 'players' in h and 'matchlogs' not in h]
    return [f"https://fbref.com{h}" for h in hrefs]

def get_player_data(session, player_url):
    # Fetch and parse the page
    soup = get_soup(session, player_url)
    if not soup:
        print(f"Failed to retrieve page for {player_url}")
        return None

    # Extract player name
    player_name = None
    h1_tag = soup.find("h1", {"itemprop": "name"}) or soup.find("h1")
    player_name = h1_tag.get_text(strip=True) if h1_tag else "Unknown"

    try:
        # Extract player stats table
        tables = pd.read_html(str(soup), match=player_name)
        if not tables:
            print(f"No matching table found for {player_name}.")
            return None

        df = tables[0]  # Assume first table is relevant

        # Extract minutes played
        minutes = None
        footer_div = soup.find("div", id=re.compile("^tfooter_scout_summary_"))
        if footer_div:
            strong_tag = footer_div.find("strong")
            if strong_tag:
                minutes_text = strong_tag.get_text(strip=True)
                minutes = int(minutes_text.split()[0])  # Extract numeric value safely

        # Add "Minutes Played" as a new row if minutes were found
        if minutes is not None:
            print(f"Found {minutes} minutes for {player_name}")
            new_row = pd.DataFrame({"Stat": ["Minutes Played"], "Value": [minutes]})
            df = pd.concat([df, new_row], ignore_index=True)

        return df

    except ValueError as e:
        print(f"Error processing {player_name}: {e}")
        return None
    
def convert_csv(df):
    return df.to_csv(index=False)

team_names = [] #To use later on for scraping associated ELO scores
years = ['2017-2018', '2018-2019', '2019-2020', '2020-2021', '2021-2022', '2022-2023', '2023-2024', '2024-2025']
def main():
    team_data = {}
    league_table_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
    
    soup_league = get_soup(s, league_table_url)
    if not soup_league:
        return
    
    team_urls = get_team_links(soup_league)
    
    # Example: just scrape first few teams to limit requests
    # Create one list per team to store player data
    
    for team_url in team_urls[:2]:
        team_name = team_url.split("/")[-1].replace("-", " ")
        clean_team_name = team_name.replace(" Stats", "")

        print(f"Scraping {team_name}")
        print(f"clean team name {clean_team_name}")
        team_names.append(clean_team_name)

        team_data[team_name] = {'players': [], 'keepers': []}   
        player_urls = get_player_links(s, team_url)
        for purl in player_urls:
            print(f"Scraping {purl}")
            df = get_player_data(s, purl)
            if df is not None:
                if df.iloc[0, 0] == "PSxG-GA":
                    team_data[team_name]['keepers'].append(df)
                else:
                    team_data[team_name]['players'].append(df)
            else:
                print(f"No data for {purl}")
            # random sleep
            time.sleep(random.uniform(2, 5))
    
    print(f"Scraped data for {len(team_data)} teams.")
    print("Data keys:", team_data.keys())
    print("Data for first team:", team_data['Liverpool Stats'])



    # Convert each team's data to a single CSV for players and another for keepers
    for team, data in team_data.items():
        for key, dfs in data.items():
            if dfs:  # Ensure there is data to concatenate
                combined_df = pd.concat(dfs, ignore_index=True)  # Combine all DataFrames
                print(f"\n===== {team} - {key.upper()} =====")
                display(combined_df)
                csv = convert_csv(combined_df)  # Convert to CSV format
                filename = f"{team}_{key}.csv"
                
                with open(filename, "w") as f:
                    f.write(csv)
                    print(f"Saved {filename}")


if __name__ == "__main__":
    main()


Scraping Liverpool Stats
clean team name Liverpool
Scraping https://fbref.com/en/players/e06683ca/Virgil-van-Dijk


C:\Users\tobia\AppData\Local\Temp\ipykernel_13648\3939547987.py:78: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup), match=player_name)


Found 4296 minutes for Virgil van Dijk
Scraping https://fbref.com/en/players/e342ad68/Mohamed-Salah


C:\Users\tobia\AppData\Local\Temp\ipykernel_13648\3939547987.py:78: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(str(soup), match=player_name)


Found 3865 minutes for Mohamed Salah
Scraping https://fbref.com/en/players/b8e740fb/Ryan-Gravenberch


...

Saved Liverpool Stats_keepers.csv

===== Arsenal Stats - PLAYERS =====


,Statistic,Per 90,Percentile,Stat,Value
0,Non-Penalty Goals,0.04,56.0,NaN,NaN
1,npxG: Non-Penalty xG,0.06,65.0,NaN,NaN
2,Shots Total,0.20,9.0,NaN,NaN
3,Assists,0.00,27.0,NaN,NaN
4,xAG: Exp. Assisted Goals,0.01,37.0,NaN,NaN
...,...,...,...,...,...
523,Interceptions,0.26,19.0,NaN,NaN
524,Blocks,0.77,38.0,NaN,NaN
525,Clearances,0.90,82.0,NaN,NaN
526,Aerials Won,1.93,97.0,NaN,NaN


Saved Arsenal Stats_players.csv

===== Arsenal Stats - KEEPERS =====


,Statistic,Per 90,Percentile,Stat,Value
0,PSxG-GA,+0.04,57.0,NaN,NaN
1,Goals Against,0.70,99.0,NaN,NaN
2,Save Percentage,78.0%,94.0,NaN,NaN
3,PSxG/SoT,0.24,5.0,NaN,NaN
4,Save% (Penalty Kicks),16.7%,57.0,NaN,NaN
5,Clean Sheet Percentage,48.9%,99.0,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,Touches,35.83,34.0,NaN,NaN
8,Launch %,34.2%,53.0,NaN,NaN
9,Goal Kicks,3.79,10.0,NaN,NaN


Saved Arsenal Stats_keepers.csv


In [1]:
# Fetch Team aggregated season data through API calls from FBRef

import requests

response = requests.post('https://fbrapi.com/generate_api_key')
api_key = response.json()['api_key']
print("API Key:", api_key)

def fetch_team_data(params):
    url = "https://fbrapi.com/team-season-stats"
    headers = {"X-API-Key": api_key}

    response = requests.get(url, params=params, headers=headers)

    return response.json()


def get_league_ids():
    url = "https://fbrapi.com/leagues"
    headers = {"X-API-Key": api_key}

    response = requests.get(url, headers=headers)
    
    # Print the raw response text
    print("Raw API Response:", response.text)

    try:
        leagues = response.json()  # Attempt to parse JSON
    except ValueError:
        print("Error: Response is not valid JSON.")
        return {}

    # Debugging: Check the actual structure of the API response
    print("Parsed JSON:", leagues)

    if isinstance(leagues, dict):
        # Check if data is inside a key like 'data' or 'leagues'
        if "data" in leagues:
            leagues = leagues["data"]
        elif "leagues" in leagues:
            leagues = leagues["leagues"]
    
    # Check the expected structure
    if not isinstance(leagues, list):
        print("Unexpected API response structure!")
        return {}

    top5_leagues = ["Premier League", "La Liga", "Bundesliga", "Serie A", "Ligue 1"]

    # Extract league IDs correctly
    league_ids = {league["name"]: league["league_id"] for league in leagues if "name" in league and league["name"] in top5_leagues}

    return league_ids

def get_teams_per_league(league_id, season):
    """Fetch all team IDs for a given league and season."""
    url = "https://fbrapi.com/teams"
    params = {"league_id": league_id, "season_id": season}
    headers = {"X-API-Key": api_key}

    response = requests.get(url, params=params, headers=headers)
    teams = response.json()

    # Return a dictionary of {team_name: team_id}
    return {team["name"]: team["team_id"] for team in teams}

top5_league_ids = get_league_ids()
print(top5_league_ids)

seasons = ["2017-2018", "2018-2019", "2019-2020", "2020-2021", "2021-2022", "2022-2023", "2023-2024"]
teams_per_league = {}

for league_name, league_id in top5_league_ids.items():
    for season in seasons:
        teams = get_teams_per_league(league_id, season)
        # Create season-specific entries for each team
        for team_name, team_id in teams.items():
            key = f"{team_name}_{season}"
            teams_per_league[key] = team_id

print(teams_per_league)






API Key: mcGxZifpo_yd6yUaUvnw3y7RtChH3K1JQfMvBOoM4GM
Raw API Response: {"message": "Internal Server Error"}

Parsed JSON: {'message': 'Internal Server Error'}
Unexpected API response structure!
{}
{}


In [9]:
import pandas as pd
import requests
import os

def fetch_clubelo_data(team_name):
    """
    Fetch ELO data for a given team from clubelo.com
    Returns a pandas DataFrame or None if the request fails
    """
    # Format team name for URL (replace spaces with hyphens, remove special characters)
    formatted_name = team_name.replace(" ", "").replace("-", "")
    url = f"http://api.clubelo.com/{formatted_name}"
    
    try:
        # Make the request
        response = requests.get(url)
        
        if response.status_code == 200:
            # Save the content to a temporary file
            temp_file = f"temp_{formatted_name}.csv"
            with open(temp_file, 'wb') as f:
                f.write(response.content)
            
            # Read the CSV into a pandas DataFrame
            df = pd.read_csv(temp_file)
            
            # Clean up the temporary file
            os.remove(temp_file)
            
            return df
        else:
            print(f"Failed to fetch data for {team_name}: Status code {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error fetching data for {team_name}: {str(e)}")
        return None

In [10]:
for team in team_names:
    df = fetch_clubelo_data(team)
    print(df)


      Rank       Club Country  Level          Elo        From          To
0      NaN  Liverpool     ENG      1  1551.140259  1946-07-07  1946-08-31
1      NaN  Liverpool     ENG      1  1561.357666  1946-09-01  1946-09-04
2      NaN  Liverpool     ENG      1  1551.169067  1946-09-05  1946-09-07
3      NaN  Liverpool     ENG      1  1559.834717  1946-09-08  1946-09-11
4      NaN  Liverpool     ENG      1  1549.396851  1946-09-12  1946-09-14
...    ...        ...     ...    ...          ...         ...         ...
5832   2.0  Liverpool     ENG      1  1990.022705  2025-02-20  2025-02-23
5833   2.0  Liverpool     ENG      1  1990.022705  2025-02-24  2025-02-26
5834   2.0  Liverpool     ENG      1  1990.022705  2025-02-27  2025-03-05
5835   2.0  Liverpool     ENG      1  1990.022705  2025-03-06  2025-03-06
5836   2.0  Liverpool     ENG      1  1990.022705  2025-03-07  2025-12-31

[5837 rows x 7 columns]
      Rank     Club Country  Level          Elo        From          To
0      NaN  Ars

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random
from IPython.display import display
import re

# 1) Create a Session with a random or fixed User-Agent
s = requests.Session()
s.headers.update({
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/109.0.0.0 Safari/537.36"
    )
})

def get_soup(session, url):
    """Fetch a URL, return BeautifulSoup or None on failure."""
    try:
        resp = session.get(url, timeout=10)
        # Check status code
        if resp.status_code == 200:
            return BeautifulSoup(resp.text, 'html.parser')
        elif resp.status_code == 429:
            print("Received 429. Too many requests. Backing off.")
            # Wait longer or do an exponential backoff
            time.sleep(60)
            return None
        else:
            print(f"Error: got status {resp.status_code} for {url}")
            return None
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None
    
def get_competition_links(allowed_links, soup):
    table = soup.find('table', class_='liste')
    if not table:
        print("Could not find table with class 'liste'")
        return []
    
    links = table.find_all('a')
    print(f"Found {len(links)} links in table")

    hrefs = [l.get('href') for l in links if l.get('href')]
    print(f"Found {len(hrefs)} hrefs")
    
    #Select only for Top 5 competitions
    competition_urls = []
    for link in links:
        if link['href'] in allowed_links:
            competition_urls.append(f"http://clubelo.com/{link['href']}")
         
    return competition_urls

def main():
    elo_url = "http://clubelo.com/"
    print(f"Fetching {elo_url}")
    
    soup_elo = get_soup(s, elo_url)
    if not soup_elo:
        print("Failed to get soup from main page")
        return
    
    allowed_links = {"/ENG", "/ESP", "/ITA", "/GER", "/FRA"}
    competition_links = get_competition_links(allowed_links, soup_elo)
    print(f"\nTotal competition links found: {len(competition_links)}")

    for link in competition_links:
        print(link)
    print(team_names)

if __name__ == "__main__":
    main()

Fetching http://clubelo.com/
Found 62 links in table
Found 62 hrefs

Total competition links found: 5
http://clubelo.com//ENG
http://clubelo.com//ESP
http://clubelo.com//ITA
http://clubelo.com//FRA
http://clubelo.com//GER


NameError: name 'team_names' is not defined